In [ ]:
%reset -f
from preprocessing import *

In [ ]:
FLAG=[np.array([0 for _ in range(len(Sorted_Boundaries[i]))]) for i in range(The_number_of_boundaries)]

Availability=Total_points
availability_set=set(Availability)

TOTAL_PATHS=np.empty((0,len(Total_points)), int)

BIG_START_TIME=time.time()

FINAL_NUMBER_OF_PATHS = 0

for ptn in range(len(POINTS_TO_START)):

    PointJustGone=POINTS_TO_START[ptn]

    PointsToGo=NextAvailablePoints(PointJustGone, Availability)

    path_record=[PointJustGone]

    flag_record=copy.deepcopy(FLAG)

    for i in range(The_number_of_boundaries):
        if PointJustGone in Sorted_Boundaries[i]:
            flag_record[i][Sorted_Boundaries[i].index(PointJustGone)]=1

    path_record=len(PointsToGo)*[path_record]
    
    for i in range(The_number_of_boundaries):
        flag_record[i]=np.array(len(PointsToGo)*[flag_record[i]])

    
    #####################################################################
    
    start_time = time.time()

    depth=0
    while True:
        
        t1=time.time()
        print(len(PointsToGo))

        
        path_record, PointsToGo, flag_record=assassinator(path_record, PointsToGo, flag_record, depth)

        path_record=np.hstack((path_record,PointsToGo.reshape(-1,1)))
        
        total_new_pts=[]

        tmp=[]

        fl_tmp=[[] for _ in range(The_number_of_boundaries)]

        for i in range(len(PointsToGo)):
            
            current_path=path_record[i]
            current_PTG=PointsToGo[i]

            avail=availability_set - set(current_path)
            avail=list(avail)
            
            new_pts=NextAvailablePoints(current_PTG, avail)
            
            del avail


            if new_pts != []:
                
                npt_len=len(new_pts)
                tmp+=[current_path]*npt_len

                total_new_pts+=new_pts

                for kk in range(The_number_of_boundaries):
                    fl_tmp[kk]+=[flag_record[kk][i]]*npt_len

            cpt=round(len(PointsToGo),-1)
            if cpt>10000 and ( i*10%cpt  == 0 ):
                print( i*100/cpt, "percent done.")
                print()    

    
        for kk in range(The_number_of_boundaries):
            fl_tmp[kk]=np.array(fl_tmp[kk])

        del PointsToGo
        del path_record
        del flag_record

        PointsToGo=total_new_pts
        path_record=tmp
        flag_record=fl_tmp

        del total_new_pts
        del tmp
        del fl_tmp
        
        t9=time.time()

        depth+=1

        print("Time (depth:",depth,") :", t9-t1)
        print()
        print("======= Current depth: %d =======" %depth)

        if depth==total_num_points-2:
            path_record=np.hstack((np.array(path_record).reshape(-1,total_num_points-1),np.array(PointsToGo).reshape(-1,1)))
            
            print()

            break

    end_time = time.time()
    print("[%d / %d] Elapsed time :" %(ptn+1,len(POINTS_TO_START)), end_time - start_time)
   
    TOTAL_PATHS=np.vstack((TOTAL_PATHS,path_record))
    
    
    print()
    print('Found paths (from the %d th starting point): ' %(ptn+1), len(path_record))
    FINAL_NUMBER_OF_PATHS = FINAL_NUMBER_OF_PATHS + len(path_record)
    print()
    print('====================================================================================')
    print()


BIG_END_TIME=time.time()
print("TOTAL ELAPSED TIME :", BIG_END_TIME - BIG_START_TIME)
print("Final number of paths:", FINAL_NUMBER_OF_PATHS)

pd.DataFrame(TOTAL_PATHS).to_excel('TRAS_BACKUP.xlsx', index=False)

In [ ]:
print('Immigration completed')